In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import pandas as pd
import os
from os import path
import sklearn.model_selection as sk

In [2]:
# Use TensorFlow v.2 with this old v.1 code.
# E.g. placeholder variables and sessions have changed in TF2.
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
tf.__version__

'2.7.0'

In [4]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

In [5]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    new_var = this_array.copy()
    rat = (this_max - this_min)/(new_var.max() - new_var.min())
    new_var = new_var * rat
    new_var -= new_var.min()
    new_var += this_min
    if set_to_int:
        return new_var.astype('uint8')
    return new_var

In [6]:
#train = pd.read_csv('/Users/amalalmansour/Desktop/New_Images/Malignancy/train.csv')
train = pd.read_csv('/Users/amalalmansour/Desktop/New_Images/Spiculation/Agreement_Binary_Rating.csv')
len(train)

1332

In [7]:
display(train)

,noduleID,InstanceID,Dataset,Binary Rating,Bin_Rating,Binary_Rating_name,Agreement,Agreement_name,Category
0,1,4,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
1,5,118,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training
2,3,134,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
3,4,143,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
4,6,175,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training
...,...,...,...,...,...,...,...,...,...
1327,2655,40771,2,0,0,Not Spiculated,2,High Radiologist Agreement,Testing
1328,2657,40780,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
1329,2656,40794,1,2,2,Unknown,0,No Radiologist Agreement,Low/None
1330,2658,40801,1,2,2,Unknown,0,No Radiologist Agreement,Low/None


In [8]:
training_data = train.sample(frac=0.8, random_state=10)
testing_data = train.drop(training_data.index)

In [9]:
print("training_data:",len(training_data))
print("testing_data:",len(testing_data))

training_data: 1066
testing_data: 266


In [10]:
train_nodIds = np.array(training_data['noduleID'])
test_nodIds = np.array(testing_data['noduleID'])

In [11]:
# collect all images for training
image_folder = '/Users/amalalmansour/Desktop/images_1332'
train_images = []
val_images = []
test_images = []
train_noduleId = []
val_noduleId = []
test_noduleId = []
print(image_folder)

for dir1 in os.listdir(image_folder):
    print(dir1)
    if not dir1.startswith('.'):
        for file in os.listdir(os.path.join(image_folder, dir1)):
            #print(file)
            noduleID = file.split('.')[0]
            if not file.startswith('.'):
                noduleID = int(noduleID)
                if noduleID in train_nodIds:
                    train_noduleId.append(noduleID)
                    temp_image = np.loadtxt(os.path.join(image_folder, dir1,file))
                    temp_image = getNormed(temp_image)
                    #enlarged_img = getSamePadding(temp_image)
                    train_images.append(temp_image)
            
                '''if noduleID in val_nodIds:
                    val_noduleId.append(noduleID)
                    temp_image = np.loadtxt(os.path.join(image_folder, dir1,file))
                    temp_image = getNormed(temp_image)
                    val_images.append(temp_image)'''
            
        
                if noduleID in test_nodIds:
                    test_noduleId.append(noduleID)
                    temp_image = np.loadtxt(os.path.join(image_folder, dir1,file))
                    temp_image = getNormed(temp_image)
                    test_images.append(temp_image)

/Users/amalalmansour/Desktop/images_1332
.DS_Store
Malignancy_2
Malignancy_5
Malignancy_4
Malignancy_3
Malignancy_1


In [12]:
print("train_images:",len(train_images))
#print("val_images:",len(val_images))
print("test_images:",len(test_images))

train_images: 1066
test_images: 266


In [13]:
tr_df = pd.DataFrame(train_noduleId,columns =['noduleID'])
print(tr_df)

      noduleID
0         1084
1         2412
2         2374
3          471
4         1907
...        ...
1061      2609
1062       904
1063      2594
1064       290
1065      2219

[1066 rows x 1 columns]


In [14]:
train_slices = pd.merge(tr_df, training_data)
display(train_slices)

,noduleID,InstanceID,Dataset,Binary Rating,Bin_Rating,Binary_Rating_name,Agreement,Agreement_name,Category
0,1084,15915,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
1,2412,36785,2,0,0,Not Spiculated,3,All Radiologist Agreement,Testing
2,2374,36253,2,0,0,Not Spiculated,2,High Radiologist Agreement,Training
3,471,6853,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training
4,1907,29039,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training
...,...,...,...,...,...,...,...,...,...
1061,2609,39936,2,0,0,Not Spiculated,3,All Radiologist Agreement,Testing
1062,904,13301,2,0,0,Not Spiculated,2,High Radiologist Agreement,Training
1063,2594,39806,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training
1064,290,4296,2,0,0,Not Spiculated,2,High Radiologist Agreement,Training


In [15]:
te_df = pd.DataFrame(test_noduleId,columns =['noduleID'])
print(te_df)

     noduleID
0         113
1        2629
2         162
3        1437
4         165
..        ...
261       641
262       293
263       722
264        25
265       642

[266 rows x 1 columns]


In [16]:
test_slices = pd.merge(te_df, testing_data)
display(test_slices)

,noduleID,InstanceID,Dataset,Binary Rating,Bin_Rating,Binary_Rating_name,Agreement,Agreement_name,Category
0,113,1483,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training
1,2629,40403,1,2,2,Unknown,0,No Radiologist Agreement,Low/None
2,162,1982,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
3,1437,21649,2,0,0,Not Spiculated,2,High Radiologist Agreement,Testing
4,165,1923,2,0,0,Not Spiculated,2,High Radiologist Agreement,Training
...,...,...,...,...,...,...,...,...,...
261,641,9218,2,0,0,Not Spiculated,2,High Radiologist Agreement,Training
262,293,4301,2,0,0,Not Spiculated,2,High Radiologist Agreement,Training
263,722,10234,1,2,2,Unknown,1,Low Radiologist Agreement,Low/None
264,25,360,2,0,0,Not Spiculated,3,All Radiologist Agreement,Training


In [17]:
x_train, x_val, y_train, y_val = sk.train_test_split(train_images,train_slices['Agreement'],test_size=0.2, random_state = 42, shuffle=True)

In [18]:
x_test = test_images
y_test_label = test_slices['Agreement']

In [19]:
train_images = np.array(x_train)
print(len(train_images))
val_images = np.array(x_val)
print(len(val_images))
test_images = np.array(x_test)
print(len(test_images))

852
214
266


In [20]:
# Reshape the data so the pixel vectors turn into arrays, and so the instance id's are also arrays
train_data = train_images.reshape(train_images.shape[0],71,71,1)  
val_data = val_images.reshape(val_images.shape[0],71,71,1)  
test_data = test_images.reshape(test_images.shape[0], 71, 71,1)

In [21]:
train_ids = np.array(y_train)
val_ids = np.array(y_val)
test_ids = np.array(y_test_label)

In [22]:
# The number of pixels in each dimension of an image.
img_size = 71

# The images are stored in one-dimensional arrays of this length.
img_size_flat = img_size*img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size,img_size)

# Number of classes, one class for each of 10 digits.
num_classes = 4

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

In [23]:
def one_hot_encoded(class_numbers, num_classes=None):
    """
    Generate the One-Hot encoded class-labels from an array of integers.
    For example, if class_number=2 and num_classes=4 then
    the one-hot encoded label is the float array: [0. 0. 1. 0.]
    :param class_numbers:
        Array of integers with class-numbers.
        Assume the integers are from zero to num_classes-1 inclusive.
    :param num_classes:
        Number of classes. If None then use max(class_numbers)+1.
    :return:
        2-dim array of shape: [len(class_numbers), num_classes]
    """

    # Find the number of classes if None is provided.
    # Assumes the lowest class-number is zero.
    if num_classes is None:
        num_classes = np.max(class_numbers) + 1

    return np.eye(num_classes, dtype=float)[class_numbers]

In [24]:
# Convert the integer class-numbers into one-hot encoded arrays.
y_train = one_hot_encoded(class_numbers=train_ids,
                                       num_classes=num_classes)
y_val = one_hot_encoded(class_numbers=val_ids,
                                     num_classes=num_classes)
y_test = one_hot_encoded(class_numbers=test_ids,
                                      num_classes=num_classes)

In [25]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [26]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [27]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

In [28]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

In [29]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [30]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

In [31]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [32]:
y_true_cls = tf.argmax(y_true, axis=1)

In [33]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

In [34]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

In [35]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 36, 36, 16) dtype=float32>

In [36]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [37]:
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 18, 18, 36) dtype=float32>

In [38]:
layer_flat, num_features = flatten_layer(layer_conv2)

In [39]:
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 11664) dtype=float32>

In [40]:
num_features

11664

In [41]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

In [42]:
layer_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [43]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

In [44]:
layer_fc2

<tf.Tensor 'add_3:0' shape=(?, 4) dtype=float32>

In [45]:
y_pred = tf.nn.softmax(layer_fc2)

In [46]:
y_pred_cls = tf.argmax(y_pred, axis=1)

In [47]:
y_pred_cls

<tf.Tensor 'ArgMax_1:0' shape=(?,) dtype=int64>

In [48]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,
                                                        labels=y_true)

In [49]:
cost = tf.reduce_mean(cross_entropy)

In [50]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [51]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [52]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [53]:
session = tf.Session()

In [54]:
session.run(tf.global_variables_initializer())

In [55]:
train_batch_size = 64

In [56]:
def random_batch(batch_size=32):
        """
        Create a random batch of training-data.
        :param batch_size: Number of images in the batch.
        :return: 3 numpy arrays (x, y, y_cls)
        """

        # Create a random index into the training-set.
        idx = np.random.randint(low=0, high=851, size=batch_size)

        # Use the index to lookup random training-data.
        x_batch = train_data[idx]
        y_batch = y_train[idx]
        y_batch_cls = train_ids[idx]

        return x_batch, y_batch, y_batch_cls

In [57]:
# Counter for total number of iterations performed so far.
total_iterations = 0

def optimize1(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch, _ = random_batch(batch_size=train_batch_size)
        x_batch = x_batch.reshape(train_batch_size, img_size_flat)
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

        # Print status every 100 iterations.
        if i % 100 == 0:
            # Calculate the accuracy on the training-set.
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # Print it.
            print(msg.format(i + 1, acc))

    # Update the total number of iterations performed.
    total_iterations += num_iterations

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [58]:
def plot_example_errors(cls_pred, correct):
    # This function is called from print_test_accuracy() below.

    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # correct is a boolean array whether the predicted class
    # is equal to the true class for each image in the test-set.

    # Negate the boolean array.
    incorrect = (correct == False)
    
    # Get the images from the test-set that have been
    # incorrectly classified.
    images = test_images[incorrect]
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = y_test_label[incorrect]
    
    # Plot the first 9 images.
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [59]:
def plot_confusion_matrix(cls_pred):
    # This is called from print_test_accuracy() below.

    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Get the true classifications for the test-set.
    cls_true = y_test_label
    
    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # Print the confusion matrix as text.
    print(cm)

    # Plot the confusion matrix as an image.
    plt.matshow(cm)

    # Make various adjustments to the plot.
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [60]:
# Split the test-set into smaller batches of this size.
test_batch_size = 266

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = 266

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = test_images[i:j, :].reshape(test_batch_size, img_size_flat)

        # Get the associated labels.
        labels = y_test[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}
        #print(feed_dict)
        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = test_ids

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [61]:
print_test_accuracy()

Accuracy on Test-Set: 32.3% (86 / 266)


<ipython-input-60-9658df643920>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cls_pred = np.zeros(shape=num_test, dtype=np.int)


In [63]:
optimize1(num_iterations=500) 

Optimization Iteration:      1, Training Accuracy:  43.8%
Optimization Iteration:    101, Training Accuracy:  68.8%
Optimization Iteration:    201, Training Accuracy:  93.8%
Optimization Iteration:    301, Training Accuracy:  98.4%
Optimization Iteration:    401, Training Accuracy: 100.0%
Time usage: 0:00:09


In [64]:
session.close()